In [3]:

import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
import MeshCat as mc 
# using JLD2
using Test
using Random
import Convex as cvx 
import ECOS 
using ProgressMeter


  Activating environment at `~/OCRL/Shapify/Project.toml`


In [4]:
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

animate_quadrotor (generic function with 1 method)

In [5]:
function get_jacobians(model, xg, ug)
    
    A = FD.jacobian(_x -> rk4(model,dynamics,_x,ug,model.dt), xg)
    B = FD.jacobian(_u -> rk4(model,dynamics,xg,_u,model.dt), ug)
    return A, B
end

function vec_from_mat(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end


function convex_mpc_controller_full(model,params,x0,idx)

    N, Q, R = params.N, params.Q, params.R

    # get slice of the relevant trajectories for the N_mpc window
    Xref = params.Xref #[idx:(idx + N_mpc - 1)]
    Uref = params.Uref #[idx:(idx + N_mpc - 2)]
    
    # create variables 
    X = cvx.Variable(params.nx,N)
    U = cvx.Variable(params.nu,N - 1)

    # cost function (tracking cost on Xref, Uref)
    cost = 0.0
    for i = 1:N-1
#         cost += 0.5*cvx.quadform(X[:,i] - Xref[i], Q)
        cost += 0.5*cvx.quadform(X[:,i], Q)
    end
    for i = 1:(N - 1)
#         
        cost += 0.5*cvx.quadform(U[:,i], R)
    end
     xn = X[:,N]
    cost += 0.5*cvx.quadform(xn,5*Q)
    prob = cvx.minimize(cost)

    # initial condition constraint
    prob.constraints += X[:,1] == x0

    # dynamics constraints
    A,B = get_jacobians(model, Xref, Uref)
    for i = 1:(N-1)

        prob.constraints += X[:,i+1] == rk4(model, dynamics, Xref, Uref, params.dt) + A*(X[:,i]) + B*(U[:,i])
       
    end

    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)

    X = vec_from_mat(X.value) 
    U = vec_from_mat(U.value) 

    return X, U
end

convex_mpc_controller_full (generic function with 1 method)

In [6]:
function fhlqr(A::Matrix, # A matrix 
               B::Matrix, # B matrix 
               Q::Matrix, # cost weight 
               R::Matrix, # cost weight 
               Qf::Matrix,# term cost weight 
               N::Int64   # horizon size 
               )::Tuple{Vector{Matrix{Float64}}, Vector{Matrix{Float64}}} # return two matrices 
        
    # check sizes of everything 
    nx,nu = size(B)
    @assert size(A) == (nx, nx)
    @assert size(Q) == (nx, nx)
    @assert size(R) == (nu, nu)
    @assert size(Qf) == (nx, nx)
        
    # instantiate S and K 
    P = [zeros(nx,nx) for i = 1:N]
    K = [zeros(nu,nx) for i = 1:N-1]
    
    # initialize S[N] with Qf 
    P[N] = deepcopy(Qf)
    
    # Ricatti 
    for k = N-1:-1:1
        # TODO 
        P_k = P[k]
        K_k = K[k]
        
        
        K_k .= inv(R + transpose(B)*P[k+1]*B)*transpose(B)*P[k+1]*A
        P_k .= Q + transpose(A)*P[k+1]*(A - B*K_k)
        
    end
    
    return P, K 
end

fhlqr (generic function with 1 method)

In [8]:
let

    
    # dynamics parameters
    nx = 12
    nu = 4
    N = 100
    dt = 0.1
    number = 4 
    
    x0 = zeros((12, number))

    for i = 1: number 
       x0[:,i] = [rand(-5:5);rand(-5:5);1.2;0;0;0.0;zeros(6)]  
    end 
    @show x0
#    
    Q = 10*diagm(ones(nx))
    R = .1*diagm(ones(nu))

    model = (mass=0.5,
            J=Diagonal([0.0023, 0.0023, 0.004]),
            gravity=[0,0,-9.81],
            L=0.1750,
            kf=1.0,
            km=0.0245,dt = dt)

 # for staright line 
    
    Xref = zeros((12, number))
    Uref = (9.81*0.5/4)*ones(nu)
    range = [-5,-4,-3,-2,-1,1,2,3,4,5]
    @show range
    
    for i = 1: number 
#        Xref[:,i] = [0;rand(-7:7);5.2;0;0;0.0;zeros(6)]
        
        Xref[:,i] = [0;range[i];5.2;0;0;0.0;zeros(6)]
        Xref[1,i] = (Xref[2,i]*Xref[2,i])/8
    end 
#     
    @show Xref
    N_mpc = N

    u_min = zeros(nu)
    u_max = 10*ones(nu)
    x_min = -1e3*ones(nx)
    x_max = 1e3*ones(nx)
    
    N_sim = N
    
    Xsim1 =[deepcopy(x0[:,1]) for i = 1:N_sim] 
    Xsim2 =[deepcopy(x0[:,2]) for i = 1:N_sim]
    Xsim3 =[deepcopy(x0[:,3]) for i = 1:N_sim]
    Xsim4 =[deepcopy(x0[:,4]) for i = 1:N_sim]
#     Xsim5 =[deepcopy(x0[:,5]) for i = 1:N_sim]
#     Xsim6 =[deepcopy(x0[:,6]) for i = 1:N_sim]
#     Xsim7 =[deepcopy(x0[:,7]) for i = 1:N_sim]
#     Xsim8 =[deepcopy(x0[:,8]) for i = 1:N_sim]
#     Xsim9 =[deepcopy(x0[:,9]) for i = 1:N_sim]
#     Xsim10 =[deepcopy(x0[:,10]) for i = 1:N_sim]
    Xsim = [Xsim1 Xsim2 Xsim3 Xsim4] # Xsim5 Xsim6 Xsim7 Xsim8 Xsim9 Xsim10]
     
#     
    @show size(Xsim)
    params = (N = N, dt = dt, Q = Q, R = R, Xref = Xref, Uref = Uref,#idx = idx,
        x_min = x_min, x_max = x_max, u_min = u_min, u_max = u_max,nx = nx, nu = nu)
    
    for idx = 1:number
        params = (N = N, dt = dt, Q = Q, R = R, Xref = Xref[:,idx], Uref = Uref,#idx = idx,
        x_min = x_min, x_max = x_max, u_min = u_min, u_max = u_max,nx = nx, nu = nu)
        
        X_pair = Xsim
        Usim = [zeros(2) for i = 1:(N_sim-1)]
        
#         Xcvx,Ucvx = convex_mpc_controller_full(model,params,Xsim[1,idx],idx)
        A,B = get_jacobians(model, Xref[:,idx], Uref)
        P, K = fhlqr(A,B,Q,R,5*Q,N)
        
        for k = 1:N-1
        
#         Xsim[k+1,idx] = rk4(model, dynamics, Xsim[k,idx], Ucvx[k], dt)
        u_lqr = - K[k]*(Xsim[k,idx] - Xref[:,idx])
        Xsim[k+1,idx] = rk4(model, dynamics, Xsim[k,idx], u_lqr, dt)
#         @show Xsim[k+1,idx]
        end 
    end
    
    X_sim_new = [zeros(12 * number) for i = 1:N_sim]
    
    
    for i = 1:(N_sim-1)
        X_sim_new[i] = [Xsim[i, 1]; Xsim[i, 2]; Xsim[i, 3]; Xsim[i, 4]] # ; Xsim[i,5]; Xsim[i,6] ; Xsim[i,7]; Xsim[i,8]; Xsim[i,9];Xsim[i,10]]
    end
    
#     display(animate_quadrotor(number, X_sim_new, params.dt))

end

x0 = [-2.0 5.0 -4.0 -4.0; 1.0 -2.0 4.0 -1.0; 1.2 1.2 1.2 1.2; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
range = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]
Xref = [3.125 2.0 1.125 0.5; -5.0 -4.0 -3.0 -2.0; 5.2 5.2 5.2 5.2; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
size(Xsim) = (100, 4)


In [ ]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function convex_mpc_controller_deltas(model,params,x0,idx)

    N_mpc, Q, R = params.N_mpc, params.Q, params.R

    X̄ =       params.X̄[idx:(idx + N_mpc - 1)]
    Ū =       params.Ū[idx:(idx + N_mpc - 2)]
    Xref = params.Xref[idx:(idx + N_mpc - 1)]
    Uref = params.Uref[idx:(idx + N_mpc - 2)]


    ΔX = cvx.Variable(params.nx,N_mpc)
    ΔU = cvx.Variable(params.nu,N_mpc - 1)


    cost = 0.0
    for i = 1:N_mpc
        xi = X̄[i] + ΔX[:,i]
        cost += 0.5*cvx.quadform(xi - Xref[i], Q)
    end
    for i = 1:(N_mpc - 1)
        ui = Ū[i] + ΔU[:,i]
        cost += 0.5*cvx.quadform(ui - Uref[i], R)
    end

    prob = cvx.minimize(cost)

    # initial condition constraint
    prob.constraints += X̄[1] + ΔX[:,1] == x0

    # add dynamics constraints
    A,B = get_jacobians(model, X̄, Ū)
    for i = 1:(N_mpc-1)
        prob.constraints += X̄[i+1] + ΔX[:,i+1] == rk4(model, dynamics, X̄[i], Ū[i], params.dt) + A[i]*ΔX[:,i] + B[i]*ΔU[:,i]
    end
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)
    ΔU = ΔU.value

    return Ū[1] + ΔU[:,1]
end

function simulate_drones_nmpc(no_of_drones::int64, Xref::Vector)
    nx = 12
    nu = 4
    N = 250
    dt = 0.1
    x0 = [0;0;1.2;0;0;0.0;zeros(6)]

    X̄ = [deepcopy(x0) for i = 1:N]
    Ū = [(9.81*0.5/4)*ones(nu) for i = 1:(N-1)]
    Q = 10*diagm(ones(nx))
    R = .1*diagm(ones(nu))

    model = (mass=0.5,
            J=Diagonal([0.0023, 0.0023, 0.004]),
            gravity=[0,0,-9.81],
            L=0.1750,
            kf=1.0,
            km=0.0245,dt = dt)

#     Xref = [ [5*cos(t);5*cos(t)*sin(t);1.2;zeros(9)] for t = range(-pi/2,3*pi/2 + 4*pi, length = N)]
#     for i = 1:(N-1)
#         Xref[i][4:6] = (Xref[i+1][1:3] - Xref[i][1:3])/dt
#     end
    Uref = deepcopy(Ū)

    N_mpc = 40

    u_min = zeros(nu)
    u_max = 10*ones(nu)
    x_min = -1e3*ones(nx)
    x_max = 1e3*ones(nx)

    params = (N = N, dt = dt, Q = Q, R = R, X̄ = X̄, Ū = Ū, Xref = Xref, Uref = Uref, N_mpc = N_mpc,#idx = idx,
    x_min = x_min, x_max = x_max, u_min = u_min, u_max = u_max,nx = nx, nu = nu)

    # main simulation
    N_sim = 100
    @assert N_sim < N
    Xsim = [deepcopy(x0) for i = 1:N_sim]
    Usim = [zeros(2) for i = 1:(N_sim-1)]
    @showprogress "simulating" for i = 1:(N_sim-1)
        Usim[i] = convex_mpc_controller_deltas(model,params,Xsim[i],i)

        # simulate
        Xsim[i+1] = rk4(model, dynamics, Xsim[i], Usim[i], dt)

    end
    
    Xsim_new = [zeros(24) for i = 1:N_sim]
    
    for k = 1:N_sim
        Xsim_new[k] = vcat(Xsim[k], Xsim[k], Xsim[k])
        Xsim_new[k][15] = Xsim_new[k][15] + 2
        Xsim_new[k][27] = Xsim_new[k][27] + 4
    end 
    
    display(animate_quadrotor(3, Xsim_new, params.dt))

    
end
    